<a href="https://colab.research.google.com/github/etgins/Mice_ASD_Detection/blob/main/%D7%A2%D7%95%D7%AA%D7%A7_%D7%A9%D7%9C_Transfer_learning_2nd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fetching files from drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/Shareddrives/USV_RECORDINGS

Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/Shareddrives/USV_RECORDINGS'
/content


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importing libraries

In [ ]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub
import keras
import random
from tensorflow.keras import layers, models
from keras.layers import UpSampling2D,Input,LeakyReLU,MaxPooling2D,Dropout,Lambda,add,Activation,Concatenate,Dense,Flatten,Conv2D
from keras.utils.vis_utils import plot_model
# from keras.utils import plot_model
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix


Functions Definitions

In [ ]:
def Remove_None_Data_and_lowering_Labels(data,labels):
    data = [j for i,j in enumerate(data) if j is not None] 
    labels= [j for i,j in enumerate(labels) if j is not None] 
    labels = [str(i) for i in labels]
    labels = [i.lower() for i in labels]
    return data, labels
    
def Take_Syllables(Syllable_name,Data,Labels):
    ind = Labels == Syllable_name.lower()
    Syllables = Data[ind]
    return Syllables

Loading Data

In [ ]:
#img_rows, img_cols = 64, 64 #choosing resize values
Data_united = np.load('Data_united_96_18072021.npy',allow_pickle = True)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
Data_united = np.array([scaler.fit_transform(i) for i in Data_united])

Labels_united = np.load('Labels_united_18072021.npy',allow_pickle = True)
Data_united = np.repeat(Data_united[:, :, :, np.newaxis], 3, -1)

Complex_syl = Take_Syllables('Complex',Data_united,Labels_united)
Frequency_steps_syl = Take_Syllables('Frequency steps',Data_united,Labels_united)
Chevron_syl = Take_Syllables('Chevron',Data_united,Labels_united)
Composite_syl = Take_Syllables('Composite',Data_united,Labels_united)
Noise_syl = Take_Syllables('99.0',Data_united,Labels_united)
two_syllable_syl = Take_Syllables('two-syllable',Data_united,Labels_united)
upward_syl = Take_Syllables('upward',Data_united,Labels_united)
flat_syl = Take_Syllables('flat',Data_united,Labels_united)
short_syl = Take_Syllables('short',Data_united,Labels_united)
harmonics_syl = Take_Syllables('harmonics',Data_united,Labels_united)
downward_syl = Take_Syllables('downward',Data_united,Labels_united)


FileNotFoundError: ignored

In [ ]:
complex_Labels = np.zeros((len(Complex_syl)))
freq_Labels = np.ones((len(Frequency_steps_syl)))
composite_Labels = 2*np.ones((len(Composite_syl)))
Noise_Labels = 3*np.ones((len(Noise_syl)))
#Two_Labels = 4*np.ones((len(two_syllable_syl)))

Data = np.concatenate((Complex_syl, Frequency_steps_syl, Composite_syl, Noise_syl))
Labels = np.concatenate((complex_Labels, freq_Labels, composite_Labels, Noise_Labels))

from sklearn.model_selection import train_test_split
train_images, test_images, train_labels, test_labels = train_test_split(Data, Labels, test_size=0.15, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.17, random_state=42)
train_images = train_images.astype('float32')
train_images = train_images/255
val_images = val_images.astype('float32')
val_images = val_images/255
test_images = test_images.astype('float32')
test_images = test_images/255

In [ ]:
print(len(Complex_syl))
print(len(Frequency_steps_syl))
print(len(Composite_syl))
print(len(Noise_syl))
#print(len(two_syllable_syl))
tot = len(Complex_syl) + len(Frequency_steps_syl) + len(Composite_syl) + len(Noise_syl)
Complex_weight = (1/len(Complex_syl))*(tot)/4
Frequency_weight = (1/len(Frequency_steps_syl))*(tot)/4
Composite_weight = (1/len(Composite_syl))*(tot)/4
Noise_weight = (1/len(Noise_syl))*(tot)/4
#Two_weight = (1/len(two_syllable_syl))*(tot)/5

class_weight = {0:Complex_weight, 1:Frequency_weight, 2:Composite_weight, 3:Noise_weight}
print(class_weight)

In [ ]:
feature_extractor_model = "https://tfhub.dev/google/imagenet/mobilenet_v1_100_128/feature_vector/4"

!pip install "tensorflow>=1.7.0"
!pip install tensorflow-hub
!pip install tensorflow-hub

feature_extractor_layer = hub.KerasLayer(feature_extractor_model, trainable=False)

In [ ]:
#model = tf.keras.Sequential([
#                             feature_extractor_layer,
#                             tf.keras.layers.Dense(64, activation='relu'), 
#                             tf.keras.layers.Dense(4)
#])
#model.build([None, 64, 64, 3])

inputs = Input(shape=(64, 64, 3))
x = feature_extractor_layer(inputs, training=False)
x = tf.keras.layers.Dense(512, activation='relu')(x)
#x = tf.keras.layers.Dense(128, activation='relu')(x)
#x = tf.keras.layers.Dropout(0.3)(x)
#x = tf.keras.layers.Dense(64, activation='relu')(x)
#x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
datagen = ImageDataGenerator(
        width_shift_range=0.02,
        height_shift_range=0.02,
        fill_mode='nearest')

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(train_images, train_labels, shuffle=True, validation_data=(val_images, val_labels), epochs=20, class_weight=class_weight)

NameError: ignored

In [ ]:
model.evaluate(test_images, test_labels, verbose=2)

17/17 - 2s - loss: 0.8335 - accuracy: 0.6881


[0.8335346579551697, 0.6880907416343689]

In [ ]:
feature_extractor_layer.trainable = True

model.compile(optimizer=keras.optimizers.Adam(1e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(train_images, train_labels, shuffle=True, validation_data=(val_images, val_labels), epochs=10, class_weight=class_weight)


Epoch 1/10
78/78 [==============================] - 60s 741ms/step - loss: 2.0151 - accuracy: 0.4148 - val_loss: 1.0234 - val_accuracy: 0.6863
Epoch 2/10
78/78 [==============================] - 57s 732ms/step - loss: 1.0360 - accuracy: 0.6646 - val_loss: 0.8648 - val_accuracy: 0.7196
Epoch 3/10
78/78 [==============================] - 57s 727ms/step - loss: 0.9044 - accuracy: 0.7344 - val_loss: 0.8144 - val_accuracy: 0.7392
Epoch 4/10
78/78 [==============================] - 58s 741ms/step - loss: 0.9573 - accuracy: 0.6977 - val_loss: 1.0023 - val_accuracy: 0.6176
Epoch 5/10
78/78 [==============================] - 58s 737ms/step - loss: 0.9239 - accuracy: 0.7111 - val_loss: 0.8447 - val_accuracy: 0.7294
Epoch 6/10
78/78 [==============================] - 57s 736ms/step - loss: 0.8637 - accuracy: 0.7401 - val_loss: 0.9221 - val_accuracy: 0.7176
Epoch 7/10
78/78 [==============================] - 60s 771ms/step - loss: 0.8029 - accuracy: 0.7618 - val_loss: 0.8839 - val_accuracy: 0.7000

In [ ]:
model.evaluate(test_images, test_labels, verbose=2)

17/17 - 2s - loss: 0.6877 - accuracy: 0.8053


[0.6877055168151855, 0.8052930235862732]